In [2]:
import pandas as pd

df = pd.read_parquet("train-00000-of-00001.parquet")
df.shape


(620095, 73)

In [1]:
import os
os.getcwd()

'D:\\file 2\\All_DS_Python\\TUM Python and DA'

In [2]:
os.listdir()

['.ipynb_checkpoints',
 '01_data_understanding.ipynb',
 '01_extract_subset.ipynb',
 'EDA.ipynb',
 'train-00000-of-00001.parquet',
 'xgboost_sp500_future_up.ipynb']

In [15]:
%pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install yfinance==0.2.28

     ---------------------------------------- 65.3/65.3 kB 1.7 MB/s eta 0:00:00
     -------------------------------------- 112.2/112.2 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
Note: you may need to restart the kernel to use updated packages.


In [5]:
import yfinance as yf
import pandas as pd

tickers = df['Ticker'].unique()

sector_map = {}

for ticker in tickers:
    try:
        info = yf.Ticker(ticker).info
        sector_map[ticker] = info.get('sector', 'Unknown')
    except Exception:
        sector_map[ticker] = 'Unknown'

sector_df = pd.DataFrame(
    sector_map.items(),
    columns=['Ticker', 'Sector']
)

df = pd.merge(df, sector_df, on='Ticker', how='left')


In [9]:
df['Dollar_volume'] = df['Close'] * df['Volume']

ticker_stats = df.groupby(['Sector', 'Ticker'])['Dollar_volume'].mean().reset_index()
ticker_stats = ticker_stats.sort_values(['Sector', 'Dollar_volume'], ascending=[True, False])
top_3_per_sector = ticker_stats.groupby('Sector').head(3)

selected_tickers = top_3_per_sector['Ticker'].tolist()

df_33 = df[df['Ticker'].isin(selected_tickers)].copy()

print(f"finished! {len(selected_tickers)}stocks had been selected.")

finished! 3stocks had been selected.


In [11]:
print(list(df_33.columns))

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Ticker', 'SMA_5', 'SMA_10', 'SMA_20', 'SMA_50', 'EMA_12', 'EMA_26', 'MACD', 'MACD_Signal', 'MACD_Histogram', 'RSI', 'BB_Middle', 'BB_Upper', 'BB_Lower', 'BB_Width', 'BB_Position', 'Volatility', 'Price_Change', 'Price_Change_5d', 'High_Low_Ratio', 'Open_Close_Ratio', 'Volume_SMA', 'Volume_Ratio', 'Close_lag_1', 'Close_lag_2', 'Close_lag_3', 'Close_lag_5', 'Close_lag_10', 'Volume_lag_1', 'Volume_lag_2', 'Volume_lag_3', 'Volume_lag_5', 'Volume_lag_10', 'Price_Change_lag_1', 'Price_Change_lag_2', 'Price_Change_lag_3', 'Price_Change_lag_5', 'Price_Change_lag_10', 'RSI_lag_1', 'RSI_lag_2', 'RSI_lag_3', 'RSI_lag_5', 'RSI_lag_10', 'MACD_lag_1', 'MACD_lag_2', 'MACD_lag_3', 'MACD_lag_5', 'MACD_lag_10', 'Volatility_lag_1', 'Volatility_lag_2', 'Volatility_lag_3', 'Volatility_lag_5', 'Volatility_lag_10', 'Future_Return_1d', 'Future_Up_1d', 'Future_Category_1d', 'Future_Return_5d', 'Future_Up_5d', 'Future_Category_5d', 

In [14]:
import pandas as pd
import requests  # 必须导入这个库来处理网络请求

# 1. 模拟浏览器头部，防止 403 报错
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

try:
    # 抓取数据
    print("正在从 Wikipedia 获取 S&P 500 板块信息...")
    response = requests.get(url, headers=headers, timeout=10)
    wiki_df = pd.read_html(response.text)[0]
    
    # 建立映射表 (Symbol -> Sector)
    # 注意：Wikipedia 的 Symbol 有时用 '.' (如 BRK.B)，yfinance 有时用 '-'
    # 我们做一个简单的清洗，保证匹配率
    sector_map = {str(k).replace('.', '-'): v for k, v in zip(wiki_df['Symbol'], wiki_df['GICS Sector'])}
    
    # 2. 将 Sector 合并到你的主数据 df 中
    # 假设你的 df['Ticker'] 里也可能有 '.'，统一换成 '-' 匹配
    df['Ticker_clean'] = df['Ticker'].apply(lambda x: str(x).replace('.', '-'))
    df['Sector'] = df['Ticker_clean'].map(sector_map)

    # 3. 检查匹配情况
    matched_count = df['Sector'].nunique()
    if matched_count < 2:
        print(f"警告：只匹配到了 {matched_count} 个板块，请检查 Ticker 格式。")
    else:
        print(f"成功！识别到 {matched_count} 个不同的板块。")

    # 4. 计算 Dollar Volume 并筛选 Top 3
    df['Dollar_volume'] = df['Close'] * df['Volume']
    
    # 按照 Sector 和 Ticker 分组计算平均成交额
    ticker_stats = df.groupby(['Sector', 'Ticker'])['Dollar_volume'].mean().reset_index()
    
    # 排序：板块升序，成交额降序
    ticker_stats = ticker_stats.sort_values(['Sector', 'Dollar_volume'], ascending=[True, False])
    
    # 取每个板块前 3 名
    top_3_per_sector = ticker_stats.groupby('Sector').head(3)
    
    # 最终拿到的股票代码列表
    selected_tickers = top_3_per_sector['Ticker'].tolist()
    df_33 = df[df['Ticker'].isin(selected_tickers)].copy()

    print("-" * 30)
    print(f"筛选完成！最终选出股票数量: {len(selected_tickers)}")
    print("前 5 只选出的股票示例:")
    print(top_3_per_sector.head())

except Exception as e:
    print(f"运行出错: {e}")
    print("提示：如果提示 requests not defined，请确保运行了 import requests")

正在从 Wikipedia 获取 S&P 500 板块信息...


C:\Users\chenying\AppData\Local\Temp\ipykernel_25812\3595120380.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  wiki_df = pd.read_html(response.text)[0]


成功！识别到 11 个不同的板块。
------------------------------
筛选完成！最终选出股票数量: 33
前 5 只选出的股票示例:
                    Sector Ticker  Dollar_volume
9   Communication Services   META   6.642611e+09
7   Communication Services  GOOGL   4.077402e+09
6   Communication Services   GOOG   3.197840e+09
63  Consumer Discretionary   TSLA   2.486476e+10
22  Consumer Discretionary   AMZN   9.443931e+09


In [15]:
import datetime

# 1. 整理数据：确保你的 df_33 已经包含了你需要的所有列
# 建议先按 Sector（板块）和 Date（日期）排序，这样导出的文档看起来非常整齐
if 'Date' in df_33.columns:
    df_33 = df_33.sort_values(['Sector', 'Ticker', 'Date'])
else:
    df_33 = df_33.sort_values(['Sector', 'Ticker'])

# 2. 生成文件名（带上今天的日期，防止多次运行被覆盖）
today = datetime.datetime.now().strftime('%Y%m%d')
file_name = f'SP500_Top3_By_Sector_{today}.csv'

# 3. 导出为 CSV
# index=False 表示不保存 Pandas 的行索引（就是最左边那列数字），这样文档更干净
df_33.to_csv(file_name, index=False, encoding='utf-8-sig')

print(f"导出成功！文件名为: {file_name}")
print(f"数据量: {len(df_33)} 行, 包含股票数: {df_33['Ticker'].nunique()}")

导出成功！文件名为: SP500_Top3_By_Sector_20260102.csv
数据量: 39549 行, 包含股票数: 33
